In [24]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import networkx as nx

import torch

from src.utils import haversine, radians
from dataset import DayObservationsDataset

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
legend_df = pd.read_csv('data/subway_locations_connections.csv')

def parse_string_array_frame(df, column):
    df[column] = df[column].apply(lambda x : [str(i.strip().upper()[1:-1]) for i in x[1:-1].split(',')])

def parse_float_array_frame(df, column):
    df[column] = df[column].apply(lambda x : [float(i.strip()) for i in x[1:-1].split(',')])

parse_string_array_frame(legend_df, 'dest_name')
parse_string_array_frame(legend_df, 'dest_id')
parse_float_array_frame(legend_df, 'dest_lat')
parse_float_array_frame(legend_df, 'dest_long')

origin_cols = [x for x in legend_df.columns if x.startswith('origin')]
dest_cols = [x for x in legend_df.columns if x.startswith('dest')]

legend_df[['origin_lat', 'origin_long', 'dest_lat', 'dest_long']] = \
    legend_df[['origin_lat', 'origin_long', 'dest_lat', 'dest_long']].applymap(lambda x : radians(x))

In [36]:
nodes_df = legend_df[origin_cols]
# nodes_df = nodes_df.rename(columns={'origin_lat': 'lat', 'origin_long': 'lon'})
nodes_set = set(nodes_df['origin_name'].unique())

edges = []
for i, row in legend_df.iterrows():
    for name, id, lat, lon in zip(*(row[col] for col in dest_cols)):
        if name not in nodes_set:
            nodes_df = nodes_df.append(pd.DataFrame([[name, id, lat, lon]], columns=origin_cols), ignore_index=True)
            nodes_set.add(name)

        olon, olat = row['origin_long'], row['origin_lat']
        dist = float(haversine(olon, olat, lon, lat))

        edges.append((row['origin_name'], name, {'dist' : dist}))

edges[:5]
        

[('1 AV', '3 AV', {'dist': 0.4333996170612458}),
 ('1 AV', 'BEDFORD AV', {'dist': 2.579846524429354}),
 ('103 ST', '96 ST', {'dist': 0.6985162154962519}),
 ('103 ST', 'CATHEDRAL PKWY', {'dist': 0.5189875230607629}),
 ('103 ST - CORONA PLAZA', 'JUNCTION BLVD', {'dist': 0.5806388684783266})]

In [37]:
nodes_df = nodes_df.rename(columns=lambda x : x[len('origin_'):] if x.startswith('origin_') else x)
nodes_df = nodes_df.rename(columns={'long':'lon'})
nodes_df['id'] = nodes_df['name']
nodes_df = nodes_df.set_index('name')
nodes_df

,id,lat,lon
name,,,
1 AV,1 AV,0.710889,-1.291223
103 ST,103 ST,0.712085,-1.290992
103 ST - CORONA PLAZA,103 ST - CORONA PLAZA,0.711219,-1.289147
104 ST,104 ST,0.710030,-1.288711
110 ST,110 ST,0.712007,-1.290571
...,...,...,...
WOODSIDE - 61 ST,WOODSIDE - 61 ST,0.711145,-1.289850
WORLD TRADE CENTER,WORLD TRADE CENTER,0.710569,-1.291714
YORK ST,YORK ST,0.710373,-1.291312


In [38]:
emp = pd.read_csv('data/mta_employment.csv').set_index('origin_name')
emp = emp[emp.columns[-1]].rename('labor_force_rate')
emp = emp.replace('-', None).astype(float)
nodes_df = nodes_df.join(emp)
nodes_df.head(5)

,id,lat,lon,labor_force_rate
name,,,,
1 AV,1 AV,0.710889,-1.291223,73.4
103 ST,103 ST,0.712085,-1.290992,64.1
103 ST - CORONA PLAZA,103 ST - CORONA PLAZA,0.711219,-1.289147,71.8
104 ST,104 ST,0.710030,-1.288711,62.2
110 ST,110 ST,0.712007,-1.290571,61.9


In [39]:
house_pricing = pd.read_csv('data/mta_housing_salePrice.csv').set_index('origin_name')
house_pricing.drop(columns=[x for x in house_pricing.columns if not x.startswith('Estimate')], inplace=True)
house_pricing.rename(columns= lambda x: 'housing_sales_num_' + x[len('estimate!!'):], inplace=True)
nodes_df = nodes_df.join(house_pricing)
nodes_df.head(5)

,id,lat,lon,labor_force_rate,housing_sales_num_Total:,"housing_sales_num_Total:!!Less than $10,000","housing_sales_num_Total:!!$10,000 to $14,999","housing_sales_num_Total:!!$15,000 to $19,999","housing_sales_num_Total:!!$20,000 to $24,999","housing_sales_num_Total:!!$25,000 to $29,999",...,"housing_sales_num_Total:!!$175,000 to $199,999","housing_sales_num_Total:!!$200,000 to $249,999","housing_sales_num_Total:!!$250,000 to $299,999","housing_sales_num_Total:!!$300,000 to $399,999","housing_sales_num_Total:!!$400,000 to $499,999","housing_sales_num_Total:!!$500,000 to $749,999","housing_sales_num_Total:!!$750,000 to $999,999","housing_sales_num_Total:!!$1,000,000 to $1,499,999","housing_sales_num_Total:!!$1,500,000 to $1,999,999","housing_sales_num_Total:!!$2,000,000 or more"
name,,,,,,,,,,,,,,,,,,,,,
1 AV,1 AV,0.710889,-1.291223,73.4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103 ST,103 ST,0.712085,-1.290992,64.1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103 ST - CORONA PLAZA,103 ST - CORONA PLAZA,0.711219,-1.289147,71.8,33.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,9.0,0.0,9.0,15.0,0.0,0.0,0.0
104 ST,104 ST,0.710030,-1.288711,62.2,26.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0
110 ST,110 ST,0.712007,-1.290571,61.9,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
emp = pd.read_csv('data/mta_median_income.csv').set_index('origin_name')
nodes_df = nodes_df.join(emp['Estimate!!Median income (dollars)!!FAMILIES!!Families'].rename('median_income'))
nodes_df.head(5)

,id,lat,lon,labor_force_rate,housing_sales_num_Total:,"housing_sales_num_Total:!!Less than $10,000","housing_sales_num_Total:!!$10,000 to $14,999","housing_sales_num_Total:!!$15,000 to $19,999","housing_sales_num_Total:!!$20,000 to $24,999","housing_sales_num_Total:!!$25,000 to $29,999",...,"housing_sales_num_Total:!!$200,000 to $249,999","housing_sales_num_Total:!!$250,000 to $299,999","housing_sales_num_Total:!!$300,000 to $399,999","housing_sales_num_Total:!!$400,000 to $499,999","housing_sales_num_Total:!!$500,000 to $749,999","housing_sales_num_Total:!!$750,000 to $999,999","housing_sales_num_Total:!!$1,000,000 to $1,499,999","housing_sales_num_Total:!!$1,500,000 to $1,999,999","housing_sales_num_Total:!!$2,000,000 or more",median_income
name,,,,,,,,,,,,,,,,,,,,,
1 AV,1 AV,0.710889,-1.291223,73.4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,83750.0
103 ST,103 ST,0.712085,-1.290992,64.1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,195250.0
103 ST - CORONA PLAZA,103 ST - CORONA PLAZA,0.711219,-1.289147,71.8,33.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,9.0,0.0,9.0,15.0,0.0,0.0,0.0,42328.0
104 ST,104 ST,0.710030,-1.288711,62.2,26.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,69375.0
110 ST,110 ST,0.712007,-1.290571,61.9,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33323.0


In [41]:
house_pricing = pd.read_csv('data/mta_population.csv').set_index('origin_name')
house_pricing.drop(columns=[x for x in house_pricing.columns if not x.startswith('Estimate')], inplace=True)
house_pricing.rename(columns= lambda x: x[len('estimate!!population!!'):], inplace=True)
nodes_df = nodes_df.join(house_pricing)
nodes_df.head(5)

,id,lat,lon,labor_force_rate,housing_sales_num_Total:,"housing_sales_num_Total:!!Less than $10,000","housing_sales_num_Total:!!$10,000 to $14,999","housing_sales_num_Total:!!$15,000 to $19,999","housing_sales_num_Total:!!$20,000 to $24,999","housing_sales_num_Total:!!$25,000 to $29,999",...,population!!AGE!!40 to 44 years,population!!AGE!!45 to 49 years,population!!AGE!!50 to 54 years,population!!AGE!!55 to 59 years,population!!AGE!!60 to 64 years,population!!AGE!!65 to 69 years,population!!AGE!!70 to 74 years,population!!AGE!!75 to 79 years,population!!AGE!!80 to 84 years,population!!AGE!!85 years and over
name,,,,,,,,,,,,,,,,,,,,,
1 AV,1 AV,0.710889,-1.291223,73.4,0.0,0.0,0.0,0.0,0.0,0.0,...,457.0,155.0,61.0,642.0,259.0,250.0,141.0,90.0,49.0,25.0
103 ST,103 ST,0.712085,-1.290992,64.1,0.0,0.0,0.0,0.0,0.0,0.0,...,940.0,674.0,563.0,662.0,818.0,805.0,571.0,225.0,262.0,175.0
103 ST - CORONA PLAZA,103 ST - CORONA PLAZA,0.711219,-1.289147,71.8,33.0,0.0,0.0,0.0,0.0,0.0,...,356.0,278.0,240.0,84.0,103.0,116.0,41.0,29.0,42.0,22.0
104 ST,104 ST,0.710030,-1.288711,62.2,26.0,0.0,0.0,0.0,0.0,0.0,...,169.0,227.0,188.0,184.0,151.0,155.0,106.0,25.0,11.0,33.0
110 ST,110 ST,0.712007,-1.290571,61.9,0.0,0.0,0.0,0.0,0.0,0.0,...,377.0,233.0,312.0,149.0,226.0,183.0,82.0,300.0,15.0,87.0


In [44]:
nodes_df.fillna(nodes_df.mean(), inplace=True)
nodes_df

/home/eldervald/miniconda3/envs/traffic/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


,id,lat,lon,labor_force_rate,housing_sales_num_Total:,"housing_sales_num_Total:!!Less than $10,000","housing_sales_num_Total:!!$10,000 to $14,999","housing_sales_num_Total:!!$15,000 to $19,999","housing_sales_num_Total:!!$20,000 to $24,999","housing_sales_num_Total:!!$25,000 to $29,999",...,population!!AGE!!40 to 44 years,population!!AGE!!45 to 49 years,population!!AGE!!50 to 54 years,population!!AGE!!55 to 59 years,population!!AGE!!60 to 64 years,population!!AGE!!65 to 69 years,population!!AGE!!70 to 74 years,population!!AGE!!75 to 79 years,population!!AGE!!80 to 84 years,population!!AGE!!85 years and over
name,,,,,,,,,,,,,,,,,,,,,
1 AV,1 AV,0.710889,-1.291223,73.400000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,457.000000,155.000000,61.000000,642.000000,259.000000,250.000000,141.000000,90.000000,49.000000,25.000000
103 ST,103 ST,0.712085,-1.290992,64.100000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,940.000000,674.000000,563.000000,662.000000,818.000000,805.000000,571.000000,225.000000,262.000000,175.000000
103 ST - CORONA PLAZA,103 ST - CORONA PLAZA,0.711219,-1.289147,71.800000,33.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,356.000000,278.000000,240.000000,84.000000,103.000000,116.000000,41.000000,29.000000,42.000000,22.000000
104 ST,104 ST,0.710030,-1.288711,62.200000,26.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,169.000000,227.000000,188.000000,184.000000,151.000000,155.000000,106.000000,25.000000,11.000000,33.000000
110 ST,110 ST,0.712007,-1.290571,61.900000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,377.000000,233.000000,312.000000,149.000000,226.000000,183.000000,82.000000,300.000000,15.000000,87.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WOODSIDE - 61 ST,WOODSIDE - 61 ST,0.711145,-1.289850,67.200000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,687.000000,395.000000,429.000000,375.000000,153.000000,391.000000,102.000000,354.000000,163.000000,0.000000
WORLD TRADE CENTER,WORLD TRADE CENTER,0.710569,-1.291714,78.300000,57.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,469.000000,695.000000,250.000000,289.000000,211.000000,152.000000,138.000000,98.000000,29.000000,15.000000
YORK ST,YORK ST,0.710373,-1.291312,88.000000,84.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,580.000000,400.000000,328.000000,228.000000,78.000000,77.000000,16.000000,6.000000,28.000000,0.000000


In [45]:
def make_graph_from_df(nodes_df, edges, name='TLC', directed=True):
    G = nx.Graph(directed=directed)
    G.graph['Name'] = name

    G.add_nodes_from(nodes_df.to_dict('index').items())
    G.add_edges_from(edges)

    return G

In [46]:
G = make_graph_from_df(nodes_df, edges)
G.number_of_nodes(), G.number_of_edges()

(374, 503)

In [47]:
nodes_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 374 entries, 1 AV to AQUEDUCT RACETRACK
Data columns (total 51 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   id                                                  374 non-null    object 
 1   lat                                                 374 non-null    float64
 2   lon                                                 374 non-null    float64
 3   labor_force_rate                                    374 non-null    float64
 4   housing_sales_num_Total:                            374 non-null    float64
 5   housing_sales_num_Total:!!Less than $10,000         374 non-null    float64
 6   housing_sales_num_Total:!!$10,000 to $14,999        374 non-null    float64
 7   housing_sales_num_Total:!!$15,000 to $19,999        374 non-null    float64
 8   housing_sales_num_Total:!!$20,000 to $24,999        374 non-null   

In [48]:
nx.readwrite.write_gpickle(G, 'data/network.gpickle')

In [49]:
mta = pd.read_csv('data/subway_2021_ridership.csv')
mta = mta[mta['flag'] == True]
mta['DATE_time'] = pd.to_datetime(mta['DATE_time'])
mta['DATE'] = mta['DATE_time'].dt.date
mta = mta.groupby(['DATE', 'STATION'], as_index=False)['ridership'].sum()
mta = mta[mta['STATION'].isin(nodes_set)]
mta

,DATE,STATION,ridership
0,2020-12-26,1 AV,191.0
1,2020-12-26,103 ST,317.0
3,2020-12-26,104 ST,20.0
4,2020-12-26,110 ST,239.0
5,2020-12-26,111 ST,70.0
...,...,...,...
102330,2021-12-24,WINTHROP ST,390.0
102331,2021-12-24,WOODHAVEN BLVD,1546.0
102332,2021-12-24,WOODLAWN,488.0
102334,2021-12-24,YORK ST,989.0


In [50]:
for day in mta['DATE'].unique()[:30]:
    ds = DayObservationsDataset.from_dataframe_by_day(mta, day)
    torch.save(ds, f'datasets/{day}.dat')